In [1]:
"""
Author: Alan, Chen
Email: alanchen0922@gmail.com

paper: https://arxiv.org/abs/1611.08024
"""

'\nAuthor: Alan, Chen\nEmail: alanchen0922@gmail.com\n\npaper: https://arxiv.org/abs/1611.08024\n'

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.autograd import Function
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter
from torch.autograd import Variable
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [3]:
def read_bci_data():
    S4b_train = np.load('S4b_train.npz')
    X11b_train = np.load('X11b_train.npz')
    S4b_test = np.load('S4b_test.npz')
    X11b_test = np.load('X11b_test.npz')

    train_data = np.concatenate((S4b_train['signal'], X11b_train['signal']), axis=0)
    train_label = np.concatenate((S4b_train['label'], X11b_train['label']), axis=0)
    test_data = np.concatenate((S4b_test['signal'], X11b_test['signal']), axis=0)
    test_label = np.concatenate((S4b_test['label'], X11b_test['label']), axis=0)

    train_label = train_label - 1
    test_label = test_label -1
    train_data = np.transpose(np.expand_dims(train_data, axis=1), (0, 1, 3, 2))
    test_data = np.transpose(np.expand_dims(test_data, axis=1), (0, 1, 3, 2))

    mask = np.where(np.isnan(train_data))
    train_data[mask] = np.nanmean(train_data)

    mask = np.where(np.isnan(test_data))
    test_data[mask] = np.nanmean(test_data)

    print(train_data.shape, train_label.shape, test_data.shape, test_label.shape)

    return train_data, train_label, test_data, test_label

In [4]:
class DepthwiseConv2d(torch.nn.Conv2d):
    def __init__(self,
                 in_channels,
                 depth_multiplier=1,
                 kernel_size=3,
                 stride=1,
                 padding=0,
                 dilation=1,
                 bias=True,
                 padding_mode='zeros'
                 ):
        out_channels = in_channels * depth_multiplier
        super().__init__(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            dilation=dilation,
            groups=in_channels,
            bias=bias,
            padding_mode=padding_mode
        )

In [5]:
class SeparableConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=0,dilation=1,bias=False):
        super(SeparableConv2d,self).__init__()

        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, groups=in_channels, bias=bias, padding=0)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)
    
    def forward(self,x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

In [6]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.C = 2
        self.T = 750
        self.F1 = 8
        self.D = 2
        self.F2 = 16
        self.kernel_length = 64
        self.dropoutRate = 0.5
        self.classes = 2
        
        # block 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=self.F1, kernel_size=(1, self.kernel_length), padding=0, bias=False)
        self.zeropad1 = nn.ZeroPad2d((31, 32, 0, 0))
        self.batchnorm1 = nn.BatchNorm2d(self.F1)
        self.depthwise1 = DepthwiseConv2d(self.F1, kernel_size=(self.C, 1), depth_multiplier=self.D, bias=False)
        self.batchnorm2 = nn.BatchNorm2d(self.F1*self.D)
        self.averagePooling1 = nn.AvgPool2d((1, 4))
        self.dropout1 = nn.Dropout(p=self.dropoutRate)
        
        # block 2
        self.separableconv1 = SeparableConv2d(self.F1*self.D, self.F2, kernel_size=(1, 16), bias=False, padding=0)
        self.zeropad2 = nn.ZeroPad2d((8, 7, 0, 0))
        self.batchnorm3 = nn.BatchNorm2d(self.F2)
        self.averagePooling2 = nn.AvgPool2d(kernel_size=(1,8))
        self.dropout2 = nn.Dropout(p=self.dropoutRate)
        self.flatten1 = nn.Flatten()
        
        self.dense1 = nn.Linear(self.F2*(self.T//32), self.classes)
        
        
    def forward(self, x):
        
        # block 1
#         print("input",x.shape)
        x = self.conv1(x)
#         print("conv1", x.shape)
        x = self.zeropad1(x)
#         print("zeropad", x.shape)
        x = self.batchnorm1(x)
#         print("batchnorm", x.shape)
        x = self.depthwise1(x)
#         print("depthwise", x.shape)
        x = self.batchnorm2(x)
#         print("batchnorm",x.shape)
        x = F.elu(x) #activation
        x = self.averagePooling1(x)
#         print("averagePooling1", x.shape)
        x = self.dropout1(x)
#         print("dropout1", x.shape)
        
        # block 2
        x = self.separableconv1(x)
#         print("separableconv", x.shape)
        x = self.zeropad2(x)
#         print("zeropad", x.shape)
        x = self.batchnorm3(x)
#         print("batchnorm", x.shape)
        x = F.elu(x) #activation
        x = self.averagePooling2(x)
#         print("averagePool", x.shape)
        x = self.dropout2(x)
        x = self.flatten1(x)
#         print("flatten", x.shape)
        x = self.dense1(x)
#         print("dense", x.shape)
        x = F.softmax(x, dim=1)
        
        return x
    
model = EEGNet().to(device)
print(model)

EEGNet(
  (conv1): Conv2d(1, 8, kernel_size=(1, 64), stride=(1, 1), bias=False)
  (zeropad1): ZeroPad2d(padding=(31, 32, 0, 0), value=0.0)
  (batchnorm1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (depthwise1): DepthwiseConv2d(8, 16, kernel_size=(2, 1), stride=(1, 1), groups=8, bias=False)
  (batchnorm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (averagePooling1): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
  (dropout1): Dropout(p=0.5, inplace=False)
  (separableconv1): SeparableConv2d(
    (depthwise): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), groups=16, bias=False)
    (pointwise): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
  (zeropad2): ZeroPad2d(padding=(8, 7, 0, 0), value=0.0)
  (batchnorm3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (averagePooling2): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
  (dropout2)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    avg_loss, avg_acc = 0, 0
    for batch, (x, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(x)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_loss += loss.item()
        avg_acc += (pred.argmax(1) == y).type(torch.float).sum().item()
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader.dataset)
    return 100*avg_acc, avg_loss


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return 100*correct, test_loss

In [ ]:
x_train, y_train, x_test, y_test = read_bci_data()

x_train = torch.from_numpy(x_train).float().to(device)
y_train = torch.from_numpy(y_train).type(torch.LongTensor).to(device)

x_test = torch.from_numpy(x_test).float().to(device)
y_test = torch.from_numpy(y_test).type(torch.LongTensor).to(device) # data type is long

train = torch.utils.data.TensorDataset(x_train,y_train)
test = torch.utils.data.TensorDataset(x_test,y_test)

batch_size = 32
epochs = 10000
learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_loader = DataLoader(
    dataset=train,
    batch_size=batch_size,
#     shuffle=True,
)

test_loader = DataLoader(
    dataset=test,
    batch_size=batch_size,
#     shuffle=True,
)
train_acc = []
train_loss = []
test_acc = []
test_loss = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    tr_acc, tr_loss = train_loop(train_loader, model, loss_fn, optimizer)
    te_acc, te_loss = test_loop(test_loader, model, loss_fn)
    train_acc.append(tr_acc)
    train_loss.append(tr_loss)
    test_acc.append(te_acc)
    test_loss.append(te_loss)
print("Done!")

(1080, 1, 2, 750) (1080,) (1080, 1, 2, 750) (1080,)
Epoch 1
-------------------------------
loss: 0.725462  [    0/ 1080]
loss: 0.659678  [  320/ 1080]
loss: 0.701754  [  640/ 1080]
loss: 0.713691  [  960/ 1080]
Test Error: 
 Accuracy: 67.9%, Avg loss: 0.623469 

Epoch 2
-------------------------------
loss: 0.734080  [    0/ 1080]
loss: 0.529089  [  320/ 1080]
loss: 0.679847  [  640/ 1080]
loss: 0.705043  [  960/ 1080]
Test Error: 
 Accuracy: 70.6%, Avg loss: 0.585543 

Epoch 3
-------------------------------
loss: 0.868109  [    0/ 1080]
loss: 0.462605  [  320/ 1080]
loss: 0.690693  [  640/ 1080]
loss: 0.656326  [  960/ 1080]
Test Error: 
 Accuracy: 70.6%, Avg loss: 0.577650 

Epoch 4
-------------------------------
loss: 0.946952  [    0/ 1080]
loss: 0.440793  [  320/ 1080]
loss: 0.643258  [  640/ 1080]
loss: 0.634846  [  960/ 1080]
Test Error: 
 Accuracy: 71.4%, Avg loss: 0.570339 

Epoch 5
-------------------------------
loss: 0.954756  [    0/ 1080]
loss: 0.434370  [  320/ 1080]


loss: 0.586644  [  640/ 1080]
loss: 0.528223  [  960/ 1080]
Test Error: 
 Accuracy: 73.0%, Avg loss: 0.568018 

Epoch 40
-------------------------------
loss: 0.924652  [    0/ 1080]
loss: 0.420824  [  320/ 1080]
loss: 0.617074  [  640/ 1080]
loss: 0.569630  [  960/ 1080]
Test Error: 
 Accuracy: 72.7%, Avg loss: 0.570534 

Epoch 41
-------------------------------
loss: 1.004305  [    0/ 1080]
loss: 0.410624  [  320/ 1080]
loss: 0.571250  [  640/ 1080]
loss: 0.535668  [  960/ 1080]
Test Error: 
 Accuracy: 73.0%, Avg loss: 0.565416 

Epoch 42
-------------------------------
loss: 0.966127  [    0/ 1080]
loss: 0.402997  [  320/ 1080]
loss: 0.591438  [  640/ 1080]
loss: 0.532155  [  960/ 1080]
Test Error: 
 Accuracy: 74.2%, Avg loss: 0.557018 

Epoch 43
-------------------------------
loss: 0.980790  [    0/ 1080]
loss: 0.420248  [  320/ 1080]
loss: 0.569721  [  640/ 1080]
loss: 0.496241  [  960/ 1080]
Test Error: 
 Accuracy: 73.7%, Avg loss: 0.563112 

Epoch 44
---------------------------

loss: 0.423320  [  320/ 1080]
loss: 0.615724  [  640/ 1080]
loss: 0.519988  [  960/ 1080]
Test Error: 
 Accuracy: 73.7%, Avg loss: 0.557387 

Epoch 80
-------------------------------
loss: 1.011328  [    0/ 1080]
loss: 0.443439  [  320/ 1080]
loss: 0.612997  [  640/ 1080]
loss: 0.445032  [  960/ 1080]
Test Error: 
 Accuracy: 73.0%, Avg loss: 0.558760 

Epoch 81
-------------------------------
loss: 1.002076  [    0/ 1080]
loss: 0.372273  [  320/ 1080]
loss: 0.610750  [  640/ 1080]
loss: 0.525338  [  960/ 1080]
Test Error: 
 Accuracy: 74.8%, Avg loss: 0.547538 

Epoch 82
-------------------------------
loss: 0.931134  [    0/ 1080]
loss: 0.392726  [  320/ 1080]
loss: 0.520367  [  640/ 1080]
loss: 0.522990  [  960/ 1080]
Test Error: 
 Accuracy: 74.8%, Avg loss: 0.546164 

Epoch 83
-------------------------------
loss: 0.997473  [    0/ 1080]
loss: 0.405795  [  320/ 1080]
loss: 0.505462  [  640/ 1080]
loss: 0.470595  [  960/ 1080]
Test Error: 
 Accuracy: 75.2%, Avg loss: 0.552812 

Epoch 

Test Error: 
 Accuracy: 75.6%, Avg loss: 0.545771 

Epoch 118
-------------------------------
loss: 0.971267  [    0/ 1080]
loss: 0.416739  [  320/ 1080]
loss: 0.568776  [  640/ 1080]
loss: 0.469629  [  960/ 1080]
Test Error: 
 Accuracy: 74.5%, Avg loss: 0.549995 

Epoch 119
-------------------------------
loss: 0.990291  [    0/ 1080]
loss: 0.409603  [  320/ 1080]
loss: 0.561600  [  640/ 1080]
loss: 0.482548  [  960/ 1080]
Test Error: 
 Accuracy: 75.0%, Avg loss: 0.549550 

Epoch 120
-------------------------------
loss: 1.018090  [    0/ 1080]
loss: 0.392119  [  320/ 1080]
loss: 0.530154  [  640/ 1080]
loss: 0.412077  [  960/ 1080]
Test Error: 
 Accuracy: 75.2%, Avg loss: 0.546136 

Epoch 121
-------------------------------
loss: 0.912432  [    0/ 1080]
loss: 0.420447  [  320/ 1080]
loss: 0.549543  [  640/ 1080]
loss: 0.479132  [  960/ 1080]
Test Error: 
 Accuracy: 73.7%, Avg loss: 0.552357 

Epoch 122
-------------------------------
loss: 0.990022  [    0/ 1080]
loss: 0.419428  [  3

loss: 1.011624  [    0/ 1080]
loss: 0.367290  [  320/ 1080]
loss: 0.496161  [  640/ 1080]
loss: 0.381651  [  960/ 1080]
Test Error: 
 Accuracy: 74.9%, Avg loss: 0.547610 

Epoch 157
-------------------------------
loss: 0.975367  [    0/ 1080]
loss: 0.408466  [  320/ 1080]
loss: 0.641516  [  640/ 1080]
loss: 0.445762  [  960/ 1080]
Test Error: 
 Accuracy: 76.6%, Avg loss: 0.540449 

Epoch 158
-------------------------------
loss: 0.962753  [    0/ 1080]
loss: 0.423896  [  320/ 1080]
loss: 0.527453  [  640/ 1080]
loss: 0.476152  [  960/ 1080]
Test Error: 
 Accuracy: 75.1%, Avg loss: 0.544752 

Epoch 159
-------------------------------
loss: 0.948747  [    0/ 1080]
loss: 0.382687  [  320/ 1080]
loss: 0.497379  [  640/ 1080]
loss: 0.429228  [  960/ 1080]
Test Error: 
 Accuracy: 75.4%, Avg loss: 0.548734 

Epoch 160
-------------------------------
loss: 0.959518  [    0/ 1080]
loss: 0.403095  [  320/ 1080]
loss: 0.469497  [  640/ 1080]
loss: 0.419745  [  960/ 1080]
Test Error: 
 Accuracy: 

...

loss: 0.674492  [    0/ 1080]
loss: 0.352794  [  320/ 1080]
loss: 0.457729  [  640/ 1080]
loss: 0.340264  [  960/ 1080]
Test Error: 
 Accuracy: 80.9%, Avg loss: 0.497649 

Epoch 3525
-------------------------------
loss: 0.701660  [    0/ 1080]
loss: 0.314511  [  320/ 1080]
loss: 0.422478  [  640/ 1080]
loss: 0.337696  [  960/ 1080]
Test Error: 
 Accuracy: 80.4%, Avg loss: 0.502248 

Epoch 3526
-------------------------------
loss: 0.756123  [    0/ 1080]
loss: 0.331077  [  320/ 1080]
loss: 0.400919  [  640/ 1080]
loss: 0.345811  [  960/ 1080]
Test Error: 
 Accuracy: 79.4%, Avg loss: 0.511259 

Epoch 3527
-------------------------------
loss: 0.774224  [    0/ 1080]
loss: 0.385591  [  320/ 1080]
loss: 0.417559  [  640/ 1080]
loss: 0.323286  [  960/ 1080]
Test Error: 
 Accuracy: 77.9%, Avg loss: 0.523662 

Epoch 3528
-------------------------------
loss: 0.571451  [    0/ 1080]
loss: 0.348395  [  320/ 1080]
loss: 0.385796  [  640/ 1080]
loss: 0.319709  [  960/ 1080]
Test Error: 
 Accura

loss: 0.659207  [    0/ 1080]
loss: 0.334620  [  320/ 1080]
loss: 0.407667  [  640/ 1080]
loss: 0.345121  [  960/ 1080]
Test Error: 
 Accuracy: 79.9%, Avg loss: 0.509330 

Epoch 3564
-------------------------------
loss: 0.708772  [    0/ 1080]
loss: 0.352946  [  320/ 1080]
loss: 0.489199  [  640/ 1080]
loss: 0.339633  [  960/ 1080]
Test Error: 
 Accuracy: 79.4%, Avg loss: 0.509141 

Epoch 3565
-------------------------------
loss: 0.706093  [    0/ 1080]
loss: 0.374970  [  320/ 1080]
loss: 0.493280  [  640/ 1080]
loss: 0.313468  [  960/ 1080]
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.521959 

Epoch 3566
-------------------------------
loss: 0.672000  [    0/ 1080]
loss: 0.397698  [  320/ 1080]
loss: 0.470142  [  640/ 1080]
loss: 0.323556  [  960/ 1080]
Test Error: 
 Accuracy: 78.5%, Avg loss: 0.516825 

Epoch 3567
-------------------------------
loss: 0.678085  [    0/ 1080]
loss: 0.319269  [  320/ 1080]
loss: 0.432520  [  640/ 1080]
loss: 0.313579  [  960/ 1080]
Test Error: 
 Accura

loss: 0.344668  [  320/ 1080]
loss: 0.406281  [  640/ 1080]
loss: 0.335644  [  960/ 1080]
Test Error: 
 Accuracy: 79.3%, Avg loss: 0.513229 

Epoch 3602
-------------------------------
loss: 0.609349  [    0/ 1080]
loss: 0.373490  [  320/ 1080]
loss: 0.479828  [  640/ 1080]
loss: 0.372982  [  960/ 1080]
Test Error: 
 Accuracy: 79.9%, Avg loss: 0.508992 

Epoch 3603
-------------------------------
loss: 0.752778  [    0/ 1080]
loss: 0.345376  [  320/ 1080]
loss: 0.482854  [  640/ 1080]
loss: 0.382874  [  960/ 1080]
Test Error: 
 Accuracy: 79.2%, Avg loss: 0.511531 

Epoch 3604
-------------------------------
loss: 0.733018  [    0/ 1080]
loss: 0.370206  [  320/ 1080]
loss: 0.386437  [  640/ 1080]
loss: 0.313413  [  960/ 1080]
Test Error: 
 Accuracy: 80.4%, Avg loss: 0.506839 

Epoch 3605
-------------------------------
loss: 0.679517  [    0/ 1080]
loss: 0.348086  [  320/ 1080]
loss: 0.499126  [  640/ 1080]
loss: 0.328206  [  960/ 1080]
Test Error: 
 Accuracy: 80.1%, Avg loss: 0.508537 

loss: 0.415429  [  640/ 1080]
loss: 0.319580  [  960/ 1080]
Test Error: 
 Accuracy: 79.9%, Avg loss: 0.504304 

Epoch 3640
-------------------------------
loss: 0.719506  [    0/ 1080]
loss: 0.347392  [  320/ 1080]
loss: 0.470838  [  640/ 1080]
loss: 0.318620  [  960/ 1080]
Test Error: 
 Accuracy: 80.3%, Avg loss: 0.501532 

Epoch 3641
-------------------------------
loss: 0.699863  [    0/ 1080]
loss: 0.324529  [  320/ 1080]
loss: 0.443406  [  640/ 1080]
loss: 0.314797  [  960/ 1080]
Test Error: 
 Accuracy: 80.9%, Avg loss: 0.499032 

Epoch 3642
-------------------------------
loss: 0.763690  [    0/ 1080]
loss: 0.346051  [  320/ 1080]
loss: 0.460560  [  640/ 1080]
loss: 0.319839  [  960/ 1080]
Test Error: 
 Accuracy: 80.4%, Avg loss: 0.502113 

Epoch 3643
-------------------------------
loss: 0.638116  [    0/ 1080]
loss: 0.417361  [  320/ 1080]
loss: 0.410986  [  640/ 1080]
loss: 0.325024  [  960/ 1080]
Test Error: 
 Accuracy: 80.5%, Avg loss: 0.505538 

Epoch 3644
-----------------

loss: 0.386583  [  320/ 1080]
loss: 0.413294  [  640/ 1080]
loss: 0.313619  [  960/ 1080]
Test Error: 
 Accuracy: 81.5%, Avg loss: 0.492819 

Epoch 3679
-------------------------------
loss: 0.625983  [    0/ 1080]
loss: 0.338404  [  320/ 1080]
loss: 0.467207  [  640/ 1080]
loss: 0.319703  [  960/ 1080]
Test Error: 
 Accuracy: 79.8%, Avg loss: 0.506576 

Epoch 3680
-------------------------------
loss: 0.661008  [    0/ 1080]
loss: 0.350636  [  320/ 1080]
loss: 0.490076  [  640/ 1080]
loss: 0.344546  [  960/ 1080]
Test Error: 
 Accuracy: 79.1%, Avg loss: 0.510157 

Epoch 3681
-------------------------------
loss: 0.635697  [    0/ 1080]
loss: 0.347908  [  320/ 1080]
loss: 0.387732  [  640/ 1080]
loss: 0.314376  [  960/ 1080]
Test Error: 
 Accuracy: 80.0%, Avg loss: 0.507692 

Epoch 3682
-------------------------------
loss: 0.707413  [    0/ 1080]
loss: 0.358076  [  320/ 1080]
loss: 0.504272  [  640/ 1080]
loss: 0.333168  [  960/ 1080]
Test Error: 
 Accuracy: 81.3%, Avg loss: 0.495727 

In [ ]:
import matplotlib.pyplot as plt
plt.figure()

plt.title("Activation Function comparision(EEGNet)")
plt.ylabel("Accuracy(%)")
plt.xlabel("Epoch")
x = np.arange(1, epochs+1)
plt.plot(x, train_acc, label="elu_train")
plt.plot(x, test_acc, label="elu_test")

plt.show()

In [ ]:
plt.figure()

plt.title("Activation Function comparision(EEGNet)")
plt.ylabel("Loss")
plt.xlabel("Epoch")
x = np.arange(1, epochs+1)
plt.plot(x, train_loss, label="elu_train")
plt.plot(x, test_loss, label="elu_test")

plt.show()